In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [5]:
base_dir = 'C:/HW_3'
train_dir = os.path.join (base_dir, 'Train_Date')
test_dir = os.path.join (base_dir, 'Test_Date')
#Директория даннных для обучения модели
train_drum_dir = os.path.join (train_dir, 'drums')
train_gutars_dir = os.path.join (train_dir, 'guitars')
#Директория данных для тестирования модели
test_drum_dir = os.path.join (test_dir, 'drums')
test_gutars_dir = os.path.join (test_dir, 'guitars')


In [6]:
#Узнаем колл-во элементов в какждой 
total_test = len(os.listdir(test_drum_dir)) + len(os.listdir(test_gutars_dir))
total_train = len(os.listdir(train_drum_dir)) + len(os.listdir(train_gutars_dir))



In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#нормализируем каждое изображение
test_datagen = ImageDataGenerator(rescale = 1./255)

Batch_size = 32
Image_lenght = 80
train_generator = test_datagen.flow_from_directory(train_dir,  class_mode = 'binary', target_size=(Image_lenght,Image_lenght))
test_generator =  test_datagen.flow_from_directory(test_dir,  class_mode = 'binary', target_size=(80,80))

train_generator.class_indices

Found 545 images belonging to 2 classes.
Found 116 images belonging to 2 classes.


{'drums': 0, 'guitars': 1}

In [8]:
# Нейронка
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(Image_lenght, Image_lenght, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 78, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 512)         5

In [9]:
train_images, train_labels = next(train_generator)
test_images,  test_labels = next(test_generator)

In [10]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    steps_per_epoch=int(np.ceil(total_train / float(Batch_size))),
    epochs=20,
    validation_steps=int(np.ceil(total_test / float(Batch_size))),
)


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 18 steps, validate for 4 steps
Epoch 1/20
18/18 [==============================] - 16s 892ms/step - loss: 0.6066 - accuracy: 0.6000 - val_loss: 0.5172 - val_accuracy: 0.7672
Epoch 2/20
18/18 [==============================] - 12s 657ms/step - loss: 0.2142 - accuracy: 0.9119 - val_loss: 0.3143 - val_accuracy: 0.8966
Epoch 3/20
18/18 [==============================] - 12s 690ms/step - loss: 0.1080 - accuracy: 0.9560 - val_loss: 0.2373 - val_accuracy: 0.9138
Epoch 4/20
18/18 [==============================] - 12s 694ms/step - loss: 0.0885 - accuracy: 0.9670 - val_loss: 0.1066 - val_accuracy: 0.9569
Epoch 5/20
18/18 [==============================] - 12s 678ms/step - loss: 0.0927 - accuracy: 0.9596 - val_loss: 1.4406 - val_accuracy: 0.5086
Epoch 6/20
18/18 [==============================] - 12s 678ms/step - loss: 0.1433 - accuracy: 0.9339 - val_loss: 0.2226 - val_accuracy: 0.9310
Epoch 7/20
18/18 [==============================] -

In [13]:
predictions = model.predict(test_images) 

TP = 0
TN = 0
FP = 0
FN = 0
i = 0
while i < 32: 
    
    if str(test_labels[i])[0] == '1' and str(np.argmax(predictions[i])) == '1':
        TP += 1
    elif str(test_labels[i])[0] == '1' and str(np.argmax(predictions[i])) == '0':
        FN += 1
    elif str(test_labels[i])[0] == '0' and str(np.argmax(predictions[i])) == '1':
        FP += 1
    elif str(test_labels[i])[0] == '0' and str(np.argmax(predictions[i])) == '0':
        TN += 1
    i += 1
    
Accuracy = (TN + TP) / (TN + FP + FN + TP)
Precision = TP / (FP + TP)
Sensitivity = TP / (TP + FN)
Specificity = TN / (TN + FP)


print("TP:", TP)
print("TN:", TN)
print("FP:", FP)
print("FN:", FN)
print('')
print("Accuracy:", Accuracy)
print("Precision:", Precision)
print("Sensitivity:", Sensitivity)
print("Specificity:", Specificity)

TP: 11
TN: 20
FP: 0
FN: 1

Accuracy: 0.96875
Precision: 1.0
Sensitivity: 0.9166666666666666
Specificity: 1.0
